In [ ]:
import pandas as pd 
import numpy as np 
import yfinance as yf 
import seaborn as sns 
import ta 

sns.set(style="whitegrid")

In [ ]:
ticker = "ASELS.IS"

df = yf.download(ticker, start="2022-01-01")

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

df.head()

In [ ]:
from ta.momentum import RSIIndicator

rsi_calc = RSIIndicator(close=df["Close"])
df["rsi"] = rsi_calc.rsi()

df[["Close", "rsi"]].tail(10)


In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(12,4))

plt.plot(df.index, df["rsi"], label="RSI", color="blue")

plt.axhline(70, color="red", linestyle="--", label="Overbought")
plt.axhline(30, color="green", linestyle="--", label="Oversold")

plt.title(f"{ticker} RSI (14-Day)")
plt.xlabel("Date")
plt.ylabel("RSI Value")
plt.legend()
plt.show()

In [ ]:
from ta.trend import MACD

macd_calc = MACD(close=df["Close"])
df["macd"] = macd_calc.macd()
df["macd_signal"] = macd_calc.macd_signal()
df["macd_diff"] = macd_calc.macd_diff()

df[["Close", "macd", "macd_signal", "macd_diff"]].tail(10)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))

plt.plot(df.index, df["macd"], label="MACD", color="blue")
plt.plot(df.index, df["macd_signal"], label="Signal Line", color="red")

plt.bar(df.index, df["macd_diff"], label="MACD Histogram",
        color=(df["macd_diff"] > 0).map({True: "green", False: "red"}), alpha=0.5)

plt.title(f"{ticker} MACD Indicator")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend(loc = "upper left")
plt.grid(True)
plt.show()


In [ ]:
#Selecting necessary columns
df_model = df[["Close", "rsi", "macd", "macd_signal", "macd_diff"]].copy()

#Create Target Price
df_model["target"] = df_model["Close"].shift(-1)

#Delete last line
df_model.dropna(inplace=True)

df_model.tail()


In [ ]:
df_model.to_csv("df_model.csv")